In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import re
from functools import reduce

In [2]:
pd.options.display.max_columns = 50

In [77]:
df_inc = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2017_block_income_ann.csv')
df_pop = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2017_block_pop_with_ann.csv')
df_land_area = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/2010_tract_landarea_ann.csv')
df_medinc = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/ACS_17_5YR_S1901_with_ann.csv')
df_employment = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/employment_final.csv')
df_owner_renter = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/owner_renter_final.csv')
df_vacancy = pd.read_csv('~/Dropbox/CDS-2019-AlbanyHub/Census/vacancy_final.csv')

In [4]:
df_inc.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17
0,Id,Id2,Geography,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Margin of Error; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Margin of Error; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Margin of Error; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Margin of Error; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Margin of Error; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Margin of Error; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Margin of Error; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Margin of Error; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Margin of Error; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more"
1,1500000US130079601001,130079601001,"Block Group 1, Census Tract 9601, Baker County...",378,77,88,38,47,29,6,10,9,14,18,16,28,28,7,13,14,18,16,17,22,18,40,32,44,36,22,20,10,14,7,11,0,13
2,1500000US130079601002,130079601002,"Block Group 2, Census Tract 9601, Baker County...",426,71,88,59,7,10,7,11,33,25,8,10,7,9,9,10,44,41,14,21,79,46,53,39,12,14,45,36,13,17,3,5,4,6
3,1500000US130079602001,130079602001,"Block Group 1, Census Tract 9602, Baker County...",224,63,0,13,7,10,7,10,27,26,13,18,26,28,0,13,5,8,25,39,36,34,27,21,30,27,0,13,0,13,0,13,21,20
4,1500000US130079602002,130079602002,"Block Group 2, Census Tract 9602, Baker County...",293,64,55,48,0,13,10,11,23,23,44,39,24,22,14,15,0,13,18,33,30,32,5,8,36,25,0,13,0,13,34,30,0,13


In [5]:
df_inc['GEO.display-label']

0                                              Geography
1      Block Group 1, Census Tract 9601, Baker County...
2      Block Group 2, Census Tract 9601, Baker County...
3      Block Group 1, Census Tract 9602, Baker County...
4      Block Group 2, Census Tract 9602, Baker County...
5      Block Group 1, Census Tract 1, Dougherty Count...
6      Block Group 2, Census Tract 1, Dougherty Count...
7      Block Group 3, Census Tract 1, Dougherty Count...
8      Block Group 4, Census Tract 1, Dougherty Count...
9      Block Group 5, Census Tract 1, Dougherty Count...
10     Block Group 1, Census Tract 2, Dougherty Count...
11     Block Group 2, Census Tract 2, Dougherty Count...
12     Block Group 3, Census Tract 2, Dougherty Count...
13     Block Group 4, Census Tract 2, Dougherty Count...
14     Block Group 1, Census Tract 4, Dougherty Count...
15     Block Group 2, Census Tract 4, Dougherty Count...
16     Block Group 3, Census Tract 4, Dougherty Count...
17     Block Group 1, Census Tr

In [6]:
new = df_inc["GEO.display-label"].str.split(", ", expand = True)
df_inc["block"] = new[0] 
df_inc["tract"] = new[1] 
df_inc["county"] = new[2] 

In [7]:
df_inc = df_inc.drop('GEO.display-label', axis = 1)

In [8]:
l = len(df_inc['tract'])
df_inc1 = df_inc.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_inc.loc[i, 'tract']
    m = re.search(re_string, s)
    df_inc1.loc[i, 'tract'] = m.group(1)

In [9]:
l = len(df_inc['block'])
re_string = 'Block Group (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_inc.loc[i, 'block']
    m = re.search(re_string, s)
    df_inc1.loc[i, 'block'] = m.group(1)

In [10]:
tract_series = df_inc1['tract']
l = len(df_inc1['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = df_inc1['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df_inc1['tract'][i] + '00'

In [11]:
df_inc1.head(20)

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17,block,tract,county
0,Id,Id2,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Margin of Error; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Margin of Error; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Margin of Error; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Margin of Error; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Margin of Error; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Margin of Error; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Margin of Error; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Margin of Error; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Margin of Error; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more",Geography,None,None
1,1500000US130079601001,130079601001,378,77,88,38,47,29,6,10,9,14,18,16,28,28,7,13,14,18,16,17,22,18,40,32,44,36,22,20,10,14,7,11,0,13,1,960100,Baker County
2,1500000US130079601002,130079601002,426,71,88,59,7,10,7,11,33,25,8,10,7,9,9,10,44,41,14,21,79,46,53,39,12,14,45,36,13,17,3,5,4,6,2,960100,Baker County
3,1500000US130079602001,130079602001,224,63,0,13,7,10,7,10,27,26,13,18,26,28,0,13,5,8,25,39,36,34,27,21,30,27,0,13,0,13,0,13,21,20,1,960200,Baker County
4,1500000US130079602002,130079602002,293,64,55,48,0,13,10,11,23,23,44,39,24,22,14,15,0,13,18,33,30,32,5,8,36,25,0,13,0,13,34,30,0,13,2,960200,Baker County
5,1500000US130950001001,130950001001,280,124,0,13,0,13,0,13,9,18,50,64,13,21,15,23,88,94,35,56,26,43,44,66,0,13,0,13,0,13,0,13,0,13,1,100,Dougherty County
6,1500000US130950001002,130950001002,175,98,16,26,32,32,12,19,0,13,65,69,35,56,0,13,15,25,0,13,0,13,0,13,0,13,0,13,0,13,0,13,0,13,2,100,Dougherty County
7,1500000US130950001003,130950001003,329,118,121,77,0,13,28,43,62,65,16,25,25,41,27,41,19,33,0,13,31,39,0,13,0,13,0,13,0,13,0,13,0,13,3,100,Dougherty County
8,1500000US130950001004,130950001004,714,180,135,99,51,60,103,82,136,112,60,65,0,13,46,59,0,13,53,68,63,61,0,13,55,53,12,19,0,13,0,13,0,13,4,100,Dougherty County
9,1500000US130950001005,130950001005,407,140,90,77,37,37,10,16,13,22,17,21,131,122,0,13,13,21,20,34,8,18,11,19,13,23,0,13,0,13,44,66,0,13,5,100,Dougherty County


In [12]:
df_pop.head()

,GEO.id,GEO.id2,GEO.display-label,HD01_VD01,HD02_VD01
0,Id,Id2,Geography,Estimate; Total,Margin of Error; Total
1,1500000US130079601001,130079601001,"Block Group 1, Census Tract 9601, Baker County...",869,181
2,1500000US130079601002,130079601002,"Block Group 2, Census Tract 9601, Baker County...",1247,223
3,1500000US130079602001,130079602001,"Block Group 1, Census Tract 9602, Baker County...",487,149
4,1500000US130079602002,130079602002,"Block Group 2, Census Tract 9602, Baker County...",648,120


In [13]:
new1 = df_pop["GEO.display-label"].str.split(", ", expand = True)
df_pop["block"] = new1[0] 
df_pop["tract"] = new1[1] 
df_pop["county"] = new1[2] 

In [14]:
df_pop = df_pop.drop('GEO.display-label', axis = 1)

In [15]:
l = len(df_pop['tract'])
df_pop1 = df_pop.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_pop.loc[i, 'tract']
    m = re.search(re_string, s)
    df_pop1.loc[i, 'tract'] = m.group(1)

In [16]:
l = len(df_pop['block'])
re_string = 'Block Group (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_pop.loc[i, 'block']
    m = re.search(re_string, s)
    df_pop1.loc[i, 'block'] = m.group(1)

In [17]:
tract_series = df_pop1['tract']
l = len(df_pop1['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = df_pop1['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = df_pop1['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = df_pop1['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df_pop1['tract'][i] + '00'

In [18]:
df_pop1.head()

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,block,tract,county
0,Id,Id2,Estimate; Total,Margin of Error; Total,Geography,None,None
1,1500000US130079601001,130079601001,869,181,1,960100,Baker County
2,1500000US130079601002,130079601002,1247,223,2,960100,Baker County
3,1500000US130079602001,130079602001,487,149,1,960200,Baker County
4,1500000US130079602002,130079602002,648,120,2,960200,Baker County


In [19]:
df_land_area.head()

,GEO.id,GEO.id2,GEO.display-label,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,GCT_STUB.display-label.1,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402
0,Id,Id2,Geography,Target Geo Id,Target Geo Id2,Geographic area,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...
1,0500000US13095,13095,"Dougherty County, Georgia",0500000US13095,13095,Dougherty County,Dougherty County,94565,40801,334.63,5.94,328.69,287.7,124.1
2,0500000US13095,13095,"Dougherty County, Georgia",1400000US13095000100,13095000100,Dougherty County - Census Tract 1,Census Tract 1,5944,2545,3.17,0.02,3.15,1886.7,807.8
3,0500000US13095,13095,"Dougherty County, Georgia",1400000US13095000200,13095000200,Dougherty County - Census Tract 2,Census Tract 2,3123,1348,1.91,0.04,1.87,1667.0,719.5
4,0500000US13095,13095,"Dougherty County, Georgia",1400000US13095000400,13095000400,Dougherty County - Census Tract 4,Census Tract 4,6276,3302,2.90,0.05,2.84,2206.7,1161.0


In [20]:
new2 = df_land_area["GEO.display-label"].str.split(", ", expand = True)
df_land_area["county"] = new2[0] 

In [21]:
df_land_area = df_land_area.drop('GEO.display-label', axis = 1)

In [22]:
df_land_area['tract'] = df_land_area['GCT_STUB.display-label.1'] 
df_land_area = df_land_area.drop('GCT_STUB.display-label.1', axis=1)

In [23]:
df_land_area.head()

,GEO.id,GEO.id2,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402,county,tract
0,Id,Id2,Target Geo Id,Target Geo Id2,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...,Geography,Geographic area
1,0500000US13095,13095,0500000US13095,13095,Dougherty County,94565,40801,334.63,5.94,328.69,287.7,124.1,Dougherty County,Dougherty County
2,0500000US13095,13095,1400000US13095000100,13095000100,Dougherty County - Census Tract 1,5944,2545,3.17,0.02,3.15,1886.7,807.8,Dougherty County,Census Tract 1
3,0500000US13095,13095,1400000US13095000200,13095000200,Dougherty County - Census Tract 2,3123,1348,1.91,0.04,1.87,1667.0,719.5,Dougherty County,Census Tract 2
4,0500000US13095,13095,1400000US13095000400,13095000400,Dougherty County - Census Tract 4,6276,3302,2.90,0.05,2.84,2206.7,1161.0,Dougherty County,Census Tract 4


In [24]:
type(df_land_area['tract'][0])

str

In [25]:
l = len(df_land_area['tract'])
df_land_area1 = df_land_area.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(2,l):
    s = df_land_area.loc[i, 'tract']
    m = re.search(re_string, s)
    df_land_area1.loc[i, 'tract'] = m.group(1)

In [26]:
tract_series = df_land_area1['tract']
l = len(df_land_area1['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = df_land_area1['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = df_land_area1['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = df_land_area1['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df_land_area1['tract'][i] + '00'

In [27]:
df_land_area1.head(20)

,GEO.id,GEO.id2,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402,county,tract
0,Id,Id2,Target Geo Id,Target Geo Id2,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...,Geography,Geographic area
1,0500000US13095,13095,0500000US13095,13095,Dougherty County,94565,40801,334.63,5.94,328.69,287.7,124.1,Dougherty County,Dougherty County
2,0500000US13095,13095,1400000US13095000100,13095000100,Dougherty County - Census Tract 1,5944,2545,3.17,0.02,3.15,1886.7,807.8,Dougherty County,100
3,0500000US13095,13095,1400000US13095000200,13095000200,Dougherty County - Census Tract 2,3123,1348,1.91,0.04,1.87,1667.0,719.5,Dougherty County,200
4,0500000US13095,13095,1400000US13095000400,13095000400,Dougherty County - Census Tract 4,6276,3302,2.90,0.05,2.84,2206.7,1161.0,Dougherty County,400
5,0500000US13095,13095,1400000US13095000501,13095000501,Dougherty County - Census Tract 5.01,4798,2365,1.76,0.02,1.74,2764.5,1362.6,Dougherty County,501
6,0500000US13095,13095,1400000US13095000502,13095000502,Dougherty County - Census Tract 5.02,4151,1938,2.68,0.03,2.65,1565.4,730.8,Dougherty County,502
7,0500000US13095,13095,1400000US13095000600,13095000600,Dougherty County - Census Tract 6,2938,1484,1.34,0.01,1.33,2210.7,1116.6,Dougherty County,600
8,0500000US13095,13095,1400000US13095000700,13095000700,Dougherty County - Census Tract 7,1783,917,0.77,0.00,0.77,2314.3,1190.2,Dougherty County,700
9,0500000US13095,13095,1400000US13095000800,13095000800,Dougherty County - Census Tract 8,1607,778,0.73,0.02,0.71,2261.4,1094.8,Dougherty County,800


In [42]:
df_inc1['block_tract'] = df_inc1['block'] + '_' + df_inc1['tract']
df_inc1.head()

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,HD01_VD08,HD02_VD08,HD01_VD09,HD02_VD09,HD01_VD10,HD02_VD10,HD01_VD11,HD02_VD11,HD01_VD12,HD02_VD12,HD01_VD13,HD02_VD13,HD01_VD14,HD02_VD14,HD01_VD15,HD02_VD15,HD01_VD16,HD02_VD16,HD01_VD17,HD02_VD17,block,tract,county,block_tract
0,Id,Id2,Estimate; Total:,Margin of Error; Total:,"Estimate; Total: - Less than $10,000","Margin of Error; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Margin of Error; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Margin of Error; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Margin of Error; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Margin of Error; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Margin of Error; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Margin of Error; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Margin of Error; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Margin of Error; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Margin of Error; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Margin of Error; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Margin of Error; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Margin of Error; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Margin of Error; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Margin of Error; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more","Margin of Error; Total: - $200,000 or more",Geography,None,None,NaN
1,1500000US130079601001,130079601001,378,77,88,38,47,29,6,10,9,14,18,16,28,28,7,13,14,18,16,17,22,18,40,32,44,36,22,20,10,14,7,11,0,13,1,960100,Baker County,1_960100
2,1500000US130079601002,130079601002,426,71,88,59,7,10,7,11,33,25,8,10,7,9,9,10,44,41,14,21,79,46,53,39,12,14,45,36,13,17,3,5,4,6,2,960100,Baker County,2_960100
3,1500000US130079602001,130079602001,224,63,0,13,7,10,7,10,27,26,13,18,26,28,0,13,5,8,25,39,36,34,27,21,30,27,0,13,0,13,0,13,21,20,1,960200,Baker County,1_960200
4,1500000US130079602002,130079602002,293,64,55,48,0,13,10,11,23,23,44,39,24,22,14,15,0,13,18,33,30,32,5,8,36,25,0,13,0,13,34,30,0,13,2,960200,Baker County,2_960200


In [43]:
df_pop1['block_tract'] = df_pop1['block'] + '_' + df_pop1['tract']
df_pop1.head()

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,block,tract,county,block_tract
0,Id,Id2,Estimate; Total,Margin of Error; Total,Geography,None,None,NaN
1,1500000US130079601001,130079601001,869,181,1,960100,Baker County,1_960100
2,1500000US130079601002,130079601002,1247,223,2,960100,Baker County,2_960100
3,1500000US130079602001,130079602001,487,149,1,960200,Baker County,1_960200
4,1500000US130079602002,130079602002,648,120,2,960200,Baker County,2_960200


In [48]:
df_block_full = df_inc1.merge(df_pop1, on='block_tract', how='inner')
df_block_full.shape

(112, 47)

In [52]:
df_medinc.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,HC01_EST_VC02,HC01_MOE_VC02,HC02_EST_VC02,HC02_MOE_VC02,HC03_EST_VC02,HC03_MOE_VC02,HC04_EST_VC02,HC04_MOE_VC02,HC01_EST_VC03,HC01_MOE_VC03,HC02_EST_VC03,HC02_MOE_VC03,HC03_EST_VC03,HC03_MOE_VC03,...,HC04_MOE_VC15,HC01_EST_VC18,HC01_MOE_VC18,HC02_EST_VC18,HC02_MOE_VC18,HC03_EST_VC18,HC03_MOE_VC18,HC04_EST_VC18,HC04_MOE_VC18,HC01_EST_VC19,HC01_MOE_VC19,HC02_EST_VC19,HC02_MOE_VC19,HC03_EST_VC19,HC03_MOE_VC19,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20
0,Id,Id2,Geography,Households; Estimate; Total,Households; Margin of Error; Total,Families; Estimate; Total,Families; Margin of Error; Total,Married-couple families; Estimate; Total,Married-couple families; Margin of Error; Total,Nonfamily households; Estimate; Total,Nonfamily households; Margin of Error; Total,"Households; Estimate; Less than $10,000","Households; Margin of Error; Less than $10,000","Families; Estimate; Less than $10,000","Families; Margin of Error; Less than $10,000",Married-couple families; Estimate; Less than $...,Married-couple families; Margin of Error; Less...,"Nonfamily households; Estimate; Less than $10,000",Nonfamily households; Margin of Error; Less th...,"Households; Estimate; $10,000 to $14,999","Households; Margin of Error; $10,000 to $14,999","Families; Estimate; $10,000 to $14,999","Families; Margin of Error; $10,000 to $14,999","Married-couple families; Estimate; $10,000 to ...","Married-couple families; Margin of Error; $10,...",...,Nonfamily households; Margin of Error; Mean in...,Households; Estimate; PERCENT ALLOCATED - Hous...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Househ...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...,Households; Estimate; PERCENT ALLOCATED - Fami...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Family...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...,Households; Estimate; PERCENT ALLOCATED - Nonf...,Households; Margin of Error; PERCENT ALLOCATED...,Families; Estimate; PERCENT ALLOCATED - Nonfam...,Families; Margin of Error; PERCENT ALLOCATED -...,Married-couple families; Estimate; PERCENT ALL...,Married-couple families; Margin of Error; PERC...,Nonfamily households; Estimate; PERCENT ALLOCA...,Nonfamily households; Margin of Error; PERCENT...
1,1400000US13007960100,13007960100,"Census Tract 9601, Baker County, Georgia",804,99,528,69,345,78,276,94,21.9,8.2,11.0,7.5,10.1,10.2,42.8,16.1,6.7,3.9,7.4,5.1,4.9,4.2,...,11981,59.1,(X),(X),(X),(X),(X),(X),(X),(X),(X),58.1,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),59.8,(X)
2,1400000US13007960200,13007960200,"Census Tract 9602, Baker County, Georgia",517,83,311,66,253,61,206,80,10.6,9.1,1.6,2.7,0.0,13.7,24.3,20.8,1.4,1.9,0.0,11.3,0.0,13.7,...,6978,55.9,(X),(X),(X),(X),(X),(X),(X),(X),(X),57.9,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),52.9,(X)
3,1400000US13095000100,13095000100,"Census Tract 1, Dougherty County, Georgia",1905,220,911,173,244,117,994,210,19.0,6.9,17.9,9.2,0.0,14.2,23.9,10.7,6.3,4.2,3.5,4.5,0.0,14.2,...,4282,46.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),38.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),49.4,(X)
4,1400000US13095000200,13095000200,"Census Tract 2, Dougherty County, Georgia",1006,141,643,118,200,90,363,127,25.0,9.4,25.7,11.1,0.0,17.0,26.2,17.2,14.9,6.9,6.7,6.7,0.0,17.0,...,6997,45.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),43.1,(X),(X

In [54]:
new = df_medinc["GEO.display-label"].str.split(", ", expand = True)
df_medinc["tract"] = new[0] 
df_medinc["county"] = new[1] 

In [55]:
df_medinc = df_medinc.drop('GEO.display-label', axis = 1)

In [57]:
l = len(df_medinc['tract'])
df_medinc1 = df_medinc.copy()
re_string = 'Census Tract (\d+(?:.\d+)?)'
for i in range(1,l):
    s = df_medinc.loc[i, 'tract']
    m = re.search(re_string, s)
    df_medinc1.loc[i, 'tract'] = m.group(1)

In [61]:
tract_series = df_medinc1['tract']
l = len(df_medinc1['tract'])
for i in range(1,l):
    if '.' in tract_series[i]:
        s = tract_series[i].split('.')
        if len(s[0]) == 1:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 2:
            tract_series[i] = s[0] + s[1]
        elif len(s[0]) == 3:
            tract_series[i] = s[0] + s[1]
    else:
        if len(tract_series[i]) == 3:
            tract_series[i] = df_medinc1['tract'][i] + '00'
        elif len(tract_series[i]) == 2:
            tract_series[i] = df_medinc1['tract'][i] + '00'
        elif len(tract_series[i]) == 1:
            tract_series[i] = df_medinc1['tract'][i] + '00'
        elif len(tract_series[i]) == 4:
            tract_series[i] = df_medinc1['tract'][i] + '00'

In [70]:
df_land_area1.head(10)

,GEO.id,GEO.id2,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402,county,tract
0,Id,Id2,Target Geo Id,Target Geo Id2,Geographic area,Population,Housing units,Area in square miles - Total area,Area in square miles - Water area,Area in square miles - Land area,Density per square mile of land area - Population,Density per square mile of land area - Housing...,Geography,Geographic area
1,0500000US13095,13095,0500000US13095,13095,Dougherty County,94565,40801,334.63,5.94,328.69,287.7,124.1,Dougherty County,Dougherty County
2,0500000US13095,13095,1400000US13095000100,13095000100,Dougherty County - Census Tract 1,5944,2545,3.17,0.02,3.15,1886.7,807.8,Dougherty County,100
3,0500000US13095,13095,1400000US13095000200,13095000200,Dougherty County - Census Tract 2,3123,1348,1.91,0.04,1.87,1667.0,719.5,Dougherty County,200
4,0500000US13095,13095,1400000US13095000400,13095000400,Dougherty County - Census Tract 4,6276,3302,2.90,0.05,2.84,2206.7,1161.0,Dougherty County,400
5,0500000US13095,13095,1400000US13095000501,13095000501,Dougherty County - Census Tract 5.01,4798,2365,1.76,0.02,1.74,2764.5,1362.6,Dougherty County,501
6,0500000US13095,13095,1400000US13095000502,13095000502,Dougherty County - Census Tract 5.02,4151,1938,2.68,0.03,2.65,1565.4,730.8,Dougherty County,502
7,0500000US13095,13095,1400000US13095000600,13095000600,Dougherty County - Census Tract 6,2938,1484,1.34,0.01,1.33,2210.7,1116.6,Dougherty County,600
8,0500000US13095,13095,1400000US13095000700,13095000700,Dougherty County - Census Tract 7,1783,917,0.77,0.00,0.77,2314.3,1190.2,Dougherty County,700
9,0500000US13095,13095,1400000US13095000800,13095000800,Dougherty County - Census Tract 8,1607,778,0.73,0.02,0.71,2261.4,1094.8,Dougherty County,800


In [64]:
df_tract_full = df_land_area1.merge(df_medinc1, on='tract', how='inner')
df_tract_full.shape

(27, 145)

In [74]:
df_tract_full.head()

,GEO.id_x,GEO.id2_x,GCT_STUB.target-geo-id,GCT_STUB.target-geo-id2,GCT_STUB.display-label,HD01,HD02,SUBHD0301,SUBHD0302,SUBHD0303,SUBHD0401,SUBHD0402,county_x,tract,GEO.id_y,GEO.id2_y,HC01_EST_VC01,HC01_MOE_VC01,HC02_EST_VC01,HC02_MOE_VC01,HC03_EST_VC01,HC03_MOE_VC01,HC04_EST_VC01,HC04_MOE_VC01,HC01_EST_VC02,...,HC01_EST_VC18,HC01_MOE_VC18,HC02_EST_VC18,HC02_MOE_VC18,HC03_EST_VC18,HC03_MOE_VC18,HC04_EST_VC18,HC04_MOE_VC18,HC01_EST_VC19,HC01_MOE_VC19,HC02_EST_VC19,HC02_MOE_VC19,HC03_EST_VC19,HC03_MOE_VC19,HC04_EST_VC19,HC04_MOE_VC19,HC01_EST_VC20,HC01_MOE_VC20,HC02_EST_VC20,HC02_MOE_VC20,HC03_EST_VC20,HC03_MOE_VC20,HC04_EST_VC20,HC04_MOE_VC20,county_y
0,0500000US13095,13095,1400000US13095000100,13095000100,Dougherty County - Census Tract 1,5944,2545,3.17,0.02,3.15,1886.7,807.8,Dougherty County,100,1400000US13095000100,13095000100,1905,220,911,173,244,117,994,210,19.0,...,46.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),38.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),49.4,(X),Dougherty County
1,0500000US13095,13095,1400000US13095000200,13095000200,Dougherty County - Census Tract 2,3123,1348,1.91,0.04,1.87,1667.0,719.5,Dougherty County,200,1400000US13095000200,13095000200,1006,141,643,118,200,90,363,127,25.0,...,45.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),43.1,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),47.4,(X),Dougherty County
2,0500000US13095,13095,1400000US13095000400,13095000400,Dougherty County - Census Tract 4,6276,3302,2.90,0.05,2.84,2206.7,1161.0,Dougherty County,400,1400000US13095000400,13095000400,2786,200,1446,202,623,191,1340,223,11.5,...,35.5,(X),(X),(X),(X),(X),(X),(X),(X),(X),34.3,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),34.6,(X),Dougherty County
3,0500000US13095,13095,1400000US13095000501,13095000501,Dougherty County - Census Tract 5.01,4798,2365,1.76,0.02,1.74,2764.5,1362.6,Dougherty County,501,1400000US13095000501,13095000501,2281,156,1239,199,564,130,1042,186,1.3,...,44.9,(X),(X),(X),(X),(X),(X),(X),(X),(X),46.8,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),40.4,(X),Dougherty County
4,0500000US13095,13095,1400000US13095000502,13095000502,Dougherty County - Census Tract 5.02,4151,1938,2.68,0.03,2.65,1565.4,730.8,Dougherty County,502,1400000US13095000502,13095000502,1811,121,1198,107,969,103,613,97,3.8,...,37.3,(X),(X),(X),(X),(X),(X),(X),(X),(X),40.3,(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),(X),31.5,(X),Dougherty County


In [82]:
df_block_full1 = df_block_full.merge(df_employment, on='block_tract', how='inner')
df_block_full1.shape

(75, 85)

In [85]:
df_employment.head(20)

,GEO.id,GEO.id2,HD01_VD01,HD02_VD01,HD01_VD02,HD02_VD02,HD01_VD03,HD02_VD03,HD01_VD04,HD02_VD04,HD01_VD05,HD02_VD05,HD01_VD06,HD02_VD06,HD01_VD07,HD02_VD07,block,tract,county,block_tract
0,Id,Id2,Estimate; Total:,Margin of Error; Total:,Estimate; In labor force:,Margin of Error; In labor force:,Estimate; In labor force: - Civilian labor force:,Margin of Error; In labor force: - Civilian la...,Estimate; In labor force: - Civilian labor for...,Margin of Error; In labor force: - Civilian la...,Estimate; In labor force: - Civilian labor for...,Margin of Error; In labor force: - Civilian la...,Estimate; In labor force: - Armed Forces,Margin of Error; In labor force: - Armed Forces,Estimate; Not in labor force,Margin of Error; Not in labor force,Geography,NaN,NaN,NaN
1,1500000US130079601001,130079601001,684,144,350,113,350,113,350,113,0,13,0,13,334,87,1,960100.0,Baker County,1_960100
2,1500000US130079601002,130079601002,1038,170,488,124,488,124,461,126,27,22,0,13,550,156,2,960100.0,Baker County,2_960100
3,1500000US130079602001,130079602001,450,129,262,106,262,106,262,106,0,13,0,13,188,70,1,960200.0,Baker County,1_960200
4,1500000US130079602002,130079602002,507,98,219,69,219,69,213,69,6,9,0,13,288,81,2,960200.0,Baker County,2_960200
5,1500000US130950001001,130950001001,475,275,282,195,282,195,282,195,0,13,0,13,193,138,1,10000.0,Dougherty County,1_10000
6,1500000US130950001002,130950001002,304,183,164,106,164,106,138,100,26,25,0,13,140,109,2,10000.0,Dougherty County,2_10000
7,1500000US130950001003,130950001003,491,189,261,124,261,124,181,95,80,67,0,13,230,104,3,10000.0,Dougherty County,3_10000
8,1500000US130950001004,130950001004,1177,370,881,286,881,286,778,274,103,86,0,13,296,204,4,10000.0,Dougherty County,4_10000
9,1500000US130950001005,130950001005,780,309,507,257,507,257,467,242,40,52,0,13,273,131,5,10000.0,Dougherty County,5_10000
